# Alignment Algorithm

The signal is interpolated via a cubic interpolation function provided by scipy.interpolate.interp1d. The difference between the the signal and the shifted signal (signal is shifted from all values from -0.5 seconds to 0.5 seconds). Finally, the fmindbound algorithm is applied to get a precise value for the shift.

In [14]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from scipy import optimize
import os

def align (x1, y1, x2, y2):
    f1 = interpolate.interp1d(x1, y1, fill_value='extrapolate')
    f2 = interpolate.interp1d(x2, y2, fill_value='extrapolate')
    cx = np.arange(0, x1[len(x1)-1], 0.01)
    s = np.arange(-.5, .5, 0.01)
    sy = np.empty(s.size)
    for i, e in enumerate(s):
        sy[i] = np.sum(abs(f1(cx) - f2(cx + e)))
    f3 = interpolate.interp1d(s, sy, fill_value='extrapolate')
    os = optimize.fminbound(f3, -.5, .5)
    print("shift detected: " + str(os))
    if abs(os) > 0.49:
        plt.figure(figsize=(20, 5))
        plt.plot(np.arange(-.5, .5, 0.01), f3(np.arange(-.5, .5, 0.01)))
    x = np.arange(0, x1[len(x1)-1], 0.01)
    return (x, f2(x + os))

def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)
    
def save_song (path, x, y):
    fout = open(path, "w+")
    for i in range(x.size):
        fout.write(f"{x[i]} {y[i]}\n")
    fout.close()

In [15]:
def read_data (collection_name):
    global data, data_dir
    
    for algorithm in sorted(os.listdir(data_dir)):
        if algorithm in data:
            for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
                if collection != collection_name:
                    continue
                for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                    for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                        if (location[-4:] == '.txt'):
                            x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}")
                            data[algorithm][song][location] = (x, y)
def align_songs ():
    global data_dir
    
    for song in data['boersma'].keys():
        for part, pitches in data['boersma'][song].items():
            for algo in data.keys():
                if part in data[algo][song]:
                    bx, by = data['boersma'][song][part]
                    ox, oy = data[algo][song][part]
                    mx, my = align(bx, by, ox, oy)
                    data[algo][song][part] = (mx, my)
                else:
                    print(f"{song}/{part} does not exist for {algo}")
                    
def save_aligned (collection_name):
    for algo in data.keys():
        for song in data[algo].keys():
            for part in data[algo][song].keys():
                x, y = data[algo][song][part]
                c_path = f"{data_dir}{algo}/{collection_name}/{song}/{part}"
#                 try:
#                     save_song(c_path, x, y)
#                 except:
#                     print('permission error for: ', c_path, x, y)
                save_song(c_path, x, y)

### Scherbaum Mshavanadze

In [18]:
data_dir = '/Akamai/voice/data/pitches/'

songs = ["GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919", 
         "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713", 
         "GVM019_DaleKojas_DidgoriVillage_Didgori_20160707", 
         "GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819", 
         "GVM097_KristeAghsdga_LakhushdiVillage_MuradGigoGivi_20160819"]

data = {
    "boersma": {},
    "noll": {},
    "praat": {},
    "yin": {},
    "crepe": {},
    "maddox": {},
    "hermes": {}
}


for algo in data.keys():
    for full_song in songs:
        data[algo][full_song] = {}

read_data('Scherbaum Mshavanadze')
align_songs()
save_aligned('Scherbaum Mshavanadze')

shift detected: 2.0539125955565396e-15
shift detected: 7.45681495411895e-07
GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919/GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919_AHDS1M.txt does not exist for praat
shift detected: 2.7180783746998135e-08
shift detected: 1.0303453952573186e-06
shift detected: 1.3561342895124844e-06
shift detected: 0.01999991826687228
shift detected: 1.9567680809018384e-15
shift detected: -8.417516907507918e-07
GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919/GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919_AHDS2M.txt does not exist for praat
shift detected: 1.2239184115521683e-06
shift detected: 9.03228081862813e-07
shift detected: 1.5490709380162867e-06
shift detected: 0.020000452219748954
shift detected: 1.3877787807814457e-16
shift detected: 3.8656511925408196e-07
GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919/GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919_AHDS3M.txt does not exist for praat
shift detected: 1.743795639963407e-06
shift detected: -9.2065

shift detected: 2.6506574712925612e-15
shift detected: -1.4158049835656376e-06
GVM019_DaleKojas_DidgoriVillage_Didgori_20160707/GVM019_DaleKojas_DidgoriVillage_Didgori_20160707_VSOAX4.txt does not exist for praat
GVM019_DaleKojas_DidgoriVillage_Didgori_20160707/GVM019_DaleKojas_DidgoriVillage_Didgori_20160707_VSOAX4.txt does not exist for yin
GVM019_DaleKojas_DidgoriVillage_Didgori_20160707/GVM019_DaleKojas_DidgoriVillage_Didgori_20160707_VSOAX4.txt does not exist for crepe
shift detected: 1.0206296095310627e-06
shift detected: 0.02999672314304811
shift detected: 5.134781488891349e-16
shift detected: -2.1436491738180742e-07
GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819/GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819_AHDS1M.txt does not exist for praat
shift detected: 1.4028192691441411e-06
shift detected: 1.6717721273352387e-06
shift detected: 2.5623858978824917e-07
shift detected: 0.029999600444750904
shift detected: 1.4432899320127035e-15
shift detected: 3.135114425

### Teach Yourself Megrelian Songs

In [17]:
data_dir = '/Akamai/voice/data/pitches/'

songs = ["Ak'a Si Rekisho", 
         "Gepshvat Ghvini", 
         "Io _ Chkin Kiana", 
         "Mesishi Vardi", 
         "Meureme", 
         "Mi Re Sotsodali_", 
         "Mole Chit'i Gilakhe", 
         "O Da", 
         "Vojanudi Chkim Jargvals"]

data = {
    "boersma": {},
    "noll": {},
    "praat": {},
    "yin": {},
    "crepe": {},
    "maddox": {},
    "hermes": {}
}


for algo in data.keys():
    for full_song in songs:
        data[algo][full_song] = {}

read_data('Teach Yourself Megrelian Songs')
align_songs()
save_aligned('Teach Yourself Megrelian Songs')

shift detected: 9.853229343548264e-16
shift detected: 1.6076038497839442e-06
Ak'a Si Rekisho/Ak'a Si Rekisho.txt does not exist for praat
Ak'a Si Rekisho/Ak'a Si Rekisho.txt does not exist for yin
shift detected: 1.307196093365749e-06
shift detected: 1.0546743523399003e-06
shift detected: 0.16999958331764556
shift detected: 6.938893903907228e-17
shift detected: 1.4077017835668348e-07
Ak'a Si Rekisho/Ak'a Si Rekisho_AHDS1M.txt does not exist for praat
Ak'a Si Rekisho/Ak'a Si Rekisho_AHDS1M.txt does not exist for yin
shift detected: -1.468410008344408e-06
shift detected: -8.072393611652176e-07
shift detected: 0.02999704804695919
shift detected: -2.0261570199409107e-15
shift detected: 5.112317153690991e-07
Ak'a Si Rekisho/Ak'a Si Rekisho_AHDS2M.txt does not exist for praat
Ak'a Si Rekisho/Ak'a Si Rekisho_AHDS2M.txt does not exist for yin
shift detected: -2.173890806648438e-07
shift detected: 1.2344641852242899e-06
shift detected: 0.030001142372984343
shift detected: 6.938893903907228e-16


shift detected: 2.8762702303745466e-06
shift detected: -7.045205756260386e-07
shift detected: 0.029996429090488625
shift detected: -1.942890293094024e-15
shift detected: -1.1966499526304924e-06
Mole Chit'i Gilakhe/Mole Chit'i Gilakhe_AHDS2M.txt does not exist for praat
Mole Chit'i Gilakhe/Mole Chit'i Gilakhe_AHDS2M.txt does not exist for yin
shift detected: 1.8903026920288037e-06
shift detected: -3.6127749079796233e-07
shift detected: 0.02000304632281456
shift detected: 4.184152895626268e-07
Mole Chit'i Gilakhe/Mole Chit'i Gilakhe_AHDS3M.txt does not exist for noll
Mole Chit'i Gilakhe/Mole Chit'i Gilakhe_AHDS3M.txt does not exist for praat
Mole Chit'i Gilakhe/Mole Chit'i Gilakhe_AHDS3M.txt does not exist for yin
shift detected: 1.752291599067076e-06
shift detected: 2.872005730818389e-07
shift detected: 0.02999475707815365
shift detected: 1.2628786905111156e-15
shift detected: 2.307996317075604e-06
O Da/O Da.txt does not exist for praat
O Da/O Da.txt does not exist for yin
shift detecte

### Teach Yourself Gurian Songs

In [19]:
data_dir = '/Akamai/voice/data/pitches/'

songs = ["Adila-Alipasha",
"Indi-Mindi",
'Mival Guriashi (1)' ,
'Pikris Simghera',
"Alaverdi",
"K'alos Khelkhvavi",
'Mival Guriashi (2)' , 
"Sabodisho",
"Khasanbegura",     
"Mok'le Mravalzhamieri",
'Sadats Vshobilvar',
"Beri Ak'vans Epareba", 
"Lat'aris Simghera",    
"Mts'vanesa Da Ukudosa", 
"Shermanduli",
"Brevalo",             
"Manana",         
'Nanina (1)',      
"Shvidk'atsa",
"Chven-Mshvidoba",    
"Maq'ruli",               
'Nanina (2)',          
'Supris Khelkhvavi',
'Didi Khnidan',     
"Masp'indzelsa Mkhiarulsa", 
"Orira",                
"Ts'amok'ruli",
"Gakhsovs, T'urpa",
"Me-Rustveli",        
"P'at'ara Saq'varelo"]

data = {
    "boersma": {},
    "noll": {},
    "praat": {},
    "yin": {},
    "crepe": {},
    "maddox": {},
    "hermes": {}
}


for algo in data.keys():
    for full_song in songs:
        data[algo][full_song] = {}

read_data('Teach Yourself Gurian Songs')
align_songs()
save_aligned('Teach Yourself Gurian Songs')

shift detected: 2.858824288409778e-15
shift detected: 0.019998148238414434
Adila-Alipasha/Adila-Alipasha.txt does not exist for praat
Adila-Alipasha/Adila-Alipasha.txt does not exist for yin
shift detected: 0.03999552488418172
shift detected: 0.01000067381720934
shift detected: 0.029998494838978797
shift detected: 2.9837243786801082e-15
shift detected: 0.01999968254586175
Adila-Alipasha/Adila-Alipasha_AHDS1M.txt does not exist for praat
Adila-Alipasha/Adila-Alipasha_AHDS1M.txt does not exist for yin
shift detected: 0.03000412568681842
shift detected: -2.379640954788963e-07
shift detected: 0.02999837207366042
shift detected: -9.71445146547012e-17
shift detected: 0.019999677966629898
Adila-Alipasha/Adila-Alipasha_AHDS2M.txt does not exist for praat
Adila-Alipasha/Adila-Alipasha_AHDS2M.txt does not exist for yin
shift detected: 0.039998629407740766
shift detected: 0.01999930569542524
shift detected: 0.029997482781027783
shift detected: 1.5265566588595902e-16
shift detected: 0.019998620247

shift detected: -0.009997535234267663
shift detected: 0.02999830107806329
shift detected: 5.412337245047638e-16
shift detected: 0.019999086102283594
Mival Guriashi (2)/Mival Guriashi (2)_AHDS3M.txt does not exist for praat
Mival Guriashi (2)/Mival Guriashi (2)_AHDS3M.txt does not exist for yin
shift detected: 0.03999895083966023
shift detected: 0.020001088810981532
shift detected: 0.030000310626954834
shift detected: 7.216449660063518e-16
shift detected: 0.01999765887469356
Sabodisho/Sabodisho.txt does not exist for praat
Sabodisho/Sabodisho.txt does not exist for yin
shift detected: 0.030001141143159667
shift detected: 0.010002688201353223
shift detected: 0.029997585000887797
shift detected: 2.0816681711721685e-16
shift detected: 0.01999796170658114
Sabodisho/Sabodisho_AHDS1M.txt does not exist for praat
Sabodisho/Sabodisho_AHDS1M.txt does not exist for yin
shift detected: 0.029998661622281944
shift detected: -0.010001908312126011
shift detected: 0.0299995244764859
shift detected: 3.2

shift detected: 0.030001703225690433
shift detected: 8.000463646757337e-07
shift detected: 0.029998884735906867
shift detected: -2.3592239273284576e-16
shift detected: 0.01999980454167407
Mts'vanesa Da Ukudosa/Mts'vanesa Da Ukudosa_AHDS1M.txt does not exist for praat
Mts'vanesa Da Ukudosa/Mts'vanesa Da Ukudosa_AHDS1M.txt does not exist for yin
shift detected: 0.03000117568781063
shift detected: -0.009999681906392183
shift detected: 0.03000092835685194
shift detected: 2.5118795932144167e-15
shift detected: 0.020001902445141086
Mts'vanesa Da Ukudosa/Mts'vanesa Da Ukudosa_AHDS2M.txt does not exist for praat
Mts'vanesa Da Ukudosa/Mts'vanesa Da Ukudosa_AHDS2M.txt does not exist for yin
shift detected: 0.030002131245661427
shift detected: 0.02000003038092871
shift detected: 0.029997884643774734
shift detected: 6.938893903907228e-16
shift detected: 0.020001111734104533
Mts'vanesa Da Ukudosa/Mts'vanesa Da Ukudosa_AHDS3M.txt does not exist for praat
Mts'vanesa Da Ukudosa/Mts'vanesa Da Ukudosa_A

shift detected: 0.03000053890577311
shift detected: 2.7755575615628914e-15
shift detected: 0.02000116074191632
Maq'ruli/Maq'ruli.txt does not exist for praat
Maq'ruli/Maq'ruli.txt does not exist for yin
shift detected: 0.03999742127527045
shift detected: 0.019997868916649418
shift detected: 0.029997726650679474
shift detected: 2.6922908347160046e-15
shift detected: 0.020000822092172393
Maq'ruli/Maq'ruli_AHDS1M.txt does not exist for praat
Maq'ruli/Maq'ruli_AHDS1M.txt does not exist for yin
shift detected: 0.03999747468544323
shift detected: 0.010001547546242334
shift detected: 0.029998660055788783
shift detected: 1.8735013540549517e-15
shift detected: 0.020000696868671994
Maq'ruli/Maq'ruli_AHDS2M.txt does not exist for praat
Maq'ruli/Maq'ruli_AHDS2M.txt does not exist for yin
shift detected: 0.0300031740086912
shift detected: 0.020000306653311515
shift detected: 0.029999501802418484
shift detected: 1.9012569296705806e-15
shift detected: 0.020000878570865667
Maq'ruli/Maq'ruli_AHDS3M.txt

shift detected: -2.3592239273284576e-16
shift detected: 0.01999836229695607
Ts'amok'ruli/Ts'amok'ruli_AHDS3M.txt does not exist for praat
Ts'amok'ruli/Ts'amok'ruli_AHDS3M.txt does not exist for yin
shift detected: 0.030003143923469693
shift detected: 0.009997184755774576
shift detected: 0.0299969241835578
shift detected: -1.609823385706477e-15
shift detected: 0.019999108908175862
Gakhsovs, T'urpa/Gakhsovs, T'urpa.txt does not exist for praat
Gakhsovs, T'urpa/Gakhsovs, T'urpa.txt does not exist for yin
shift detected: 0.039998304970290714
shift detected: 0.010001304060096597
shift detected: 0.029999760948551616
shift detected: 2.0122792321330962e-15
shift detected: 0.019998417684732685
Gakhsovs, T'urpa/Gakhsovs, T'urpa_AHDS1M.txt does not exist for praat
Gakhsovs, T'urpa/Gakhsovs, T'urpa_AHDS1M.txt does not exist for yin
shift detected: 0.03999592390220526
shift detected: 1.5397659420049852e-06
shift detected: 0.030000056809489903
shift detected: 5.134781488891349e-16
shift detected: 0.